In [3]:
import tkinter as tk
from tkinter import messagebox, ttk
from decimal import Decimal, InvalidOperation, ROUND_HALF_UP, ROUND_HALF_EVEN, ROUND_DOWN, ROUND_UP
import re

class CalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Расширенный калькулятор")
        self.root.geometry("900x800")
        self.root.resizable(False, False)
        
        # Минимальное и максимальное значения (расширили в 1.5 раза)
        self.MIN_VALUE = Decimal('-1500000000000.0000000000')
        self.MAX_VALUE = Decimal('1500000000000.0000000000')
        
        # Переменные для хранения данных
        self.num_vars = [tk.StringVar(value="0") for _ in range(4)]
        self.operation_vars = [tk.StringVar(value="+") for _ in range(3)]
        self.result_var = tk.StringVar()
        self.rounded_result_var = tk.StringVar()
        self.rounding_method_var = tk.StringVar(value="mathematical")
        
        # Переменные для ошибок
        self.error_vars = [tk.StringVar() for _ in range(4)]
        
        # Ссылки на поля ввода
        self.num_entries = []
        
        self.setup_ui()
        self.setup_bindings()
        
    def setup_ui(self):
        # Заголовок
        title_label = tk.Label(
            self.root, 
            text="РАСШИРЕННЫЙ КАЛЬКУЛЯТОР (4 операнда)", 
            font=("Arial", 18, "bold"),
            fg="blue"
        )
        title_label.pack(pady=10)
        
        # Информация о студенте - горизонтально
        info_frame = tk.Frame(self.root)
        info_frame.pack(pady=10)
        
        student_info = [
            "ФИО: Ломова Виктория Александровна",
            "4 курс",
            "4 группа", 
            "2025 год"
        ]
        
        for i, info in enumerate(student_info):
            label = tk.Label(
                info_frame, 
                text=info,
                font=("Arial", 11),
                anchor="w"
            )
            if i < len(student_info) - 1:
                label.pack(side="left", padx=15)
            else:
                label.pack(side="left", padx=15)
        
        # Основной контейнер с горизонтальным разделением
        main_container = tk.Frame(self.root)
        main_container.pack(pady=10, padx=25, fill="both", expand=True)
        
        # ЛЕВАЯ ЧАСТЬ: Ввод чисел и операций
        left_frame = tk.Frame(main_container)
        left_frame.pack(side="left", fill="both", expand=True, padx=(0, 25))
        
        # Правая часть будет для результатов и округления
        right_frame = tk.Frame(main_container)
        right_frame.pack(side="right", fill="y")
        
        # Заголовок приоритета
        priority_frame = tk.Frame(left_frame, bg="lightyellow", relief="solid", bd=1)
        priority_frame.pack(fill="x", pady=(0, 15), padx=5)
        
        tk.Label(
            priority_frame,
            text="Порядок вычислений: 1) (2 и 3) 2) × ÷ 3) + -",
            font=("Arial", 10, "bold"),
            bg="lightyellow"
        ).pack(pady=3)
        
        # Создаем поля для 4 чисел и 3 операций - ГОРИЗОНТАЛЬНО с увеличенными полями
        input_grid = tk.Frame(left_frame)
        input_grid.pack(fill="both", expand=True)
        
        # Заголовки столбцов
        headers = ["Число", "Операция"]
        for col, header in enumerate(headers):
            tk.Label(input_grid, text=header, font=("Arial", 11, "bold"), bg="#f0f0f0").grid(
                row=0, column=col*2, padx=5, pady=5, sticky="ew"
            )
        
        # Первая строка: Число 1 и Операция 1
        row = 1
        tk.Label(input_grid, text="1:", font=("Arial", 11)).grid(
            row=row, column=0, padx=5, pady=5, sticky="e"
        )
        
        # Поле для числа 1
        num1_frame = tk.Frame(input_grid)
        num1_frame.grid(row=row, column=1, padx=5, pady=5, sticky="ew")
        
        entry1 = tk.Entry(
            num1_frame, 
            textvariable=self.num_vars[0],
            font=("Arial", 12),
            width=25
        )
        entry1.pack(side="left", fill="x", expand=True)
        self.num_entries.append(entry1)
        
        # Операция 1
        op1_combo = ttk.Combobox(
            input_grid,
            textvariable=self.operation_vars[0],
            values=["+", "-", "×", "÷"],
            state="readonly",
            width=8,
            font=("Arial", 11)
        )
        op1_combo.grid(row=row, column=2, padx=5, pady=5, sticky="w")
        
        # Вторая строка: Число 2 и Операция 2 (приоритетная)
        row = 2
        tk.Label(input_grid, text="2:", font=("Arial", 11), fg="red").grid(
            row=row, column=0, padx=5, pady=5, sticky="e"
        )
        
        # Поле для числа 2 (приоритетное)
        num2_frame = tk.Frame(input_grid)
        num2_frame.grid(row=row, column=1, padx=5, pady=5, sticky="ew")
        
        entry2 = tk.Entry(
            num2_frame, 
            textvariable=self.num_vars[1],
            font=("Arial", 12),
            width=25,
            bg="#FFF8DC"
        )
        entry2.pack(side="left", fill="x", expand=True)
        self.num_entries.append(entry2)
        
        # Операция 2 (приоритетная)
        op2_combo = ttk.Combobox(
            input_grid,
            textvariable=self.operation_vars[1],
            values=["+", "-", "×", "÷"],
            state="readonly",
            width=8,
            font=("Arial", 11),
            background="#FFF8DC"
        )
        op2_combo.grid(row=row, column=2, padx=5, pady=5, sticky="w")
        
        # Третья строка: Число 3 (приоритетное)
        row = 3
        tk.Label(input_grid, text="3:", font=("Arial", 11), fg="red").grid(
            row=row, column=0, padx=5, pady=5, sticky="e"
        )
        
        # Поле для числа 3 (приоритетное)
        num3_frame = tk.Frame(input_grid)
        num3_frame.grid(row=row, column=1, padx=5, pady=5, sticky="ew")
        
        entry3 = tk.Entry(
            num3_frame, 
            textvariable=self.num_vars[2],
            font=("Arial", 12),
            width=25,
            bg="#FFF8DC"
        )
        entry3.pack(side="left", fill="x", expand=True)
        self.num_entries.append(entry3)
        
        # Операция 3
        op3_combo = ttk.Combobox(
            input_grid,
            textvariable=self.operation_vars[2],
            values=["+", "-", "×", "÷"],
            state="readonly",
            width=8,
            font=("Arial", 11)
        )
        op3_combo.grid(row=row, column=2, padx=5, pady=5, sticky="w")
        
        # Четвертая строка: Число 4
        row = 4
        tk.Label(input_grid, text="4:", font=("Arial", 11)).grid(
            row=row, column=0, padx=5, pady=5, sticky="e"
        )
        
        # Поле для числа 4
        num4_frame = tk.Frame(input_grid)
        num4_frame.grid(row=row, column=1, padx=5, pady=5, sticky="ew")
        
        entry4 = tk.Entry(
            num4_frame, 
            textvariable=self.num_vars[3],
            font=("Arial", 12),
            width=25
        )
        entry4.pack(side="left", fill="x", expand=True)
        self.num_entries.append(entry4)
        
        # Место для операции 4 (нет операции для 4 числа)
        tk.Label(input_grid, text="", font=("Arial", 11)).grid(
            row=row, column=2, padx=5, pady=5
        )
        
        # Фрейм для сообщений об ошибках (под полями ввода)
        error_frame = tk.Frame(left_frame)
        error_frame.pack(fill="x", pady=(10, 0))
        
        # Создаем метки для ошибок
        self.error_labels = []
        for i in range(4):
            error_label = tk.Label(
                error_frame,
                textvariable=self.error_vars[i],
                font=("Arial", 9),
                fg="red",
                wraplength=250,
                height=2
            )
            error_label.grid(row=i//2, column=i%2, padx=10, pady=2, sticky="w")
            self.error_labels.append(error_label)
        
        # Кнопки внизу левой части
        button_frame = tk.Frame(left_frame)
        button_frame.pack(pady=10)
        
        tk.Button(
            button_frame,
            text="Вычислить",
            command=self.calculate,
            font=("Arial", 12, "bold"),
            bg="lightgreen",
            padx=30,
            pady=12
        ).pack(side="left", padx=15)
        
        tk.Button(
            button_frame,
            text="Очистить",
            command=self.clear_all,
            font=("Arial", 12),
            bg="lightcoral",
            padx=30,
            pady=12
        ).pack(side="left", padx=15)
        
        # ПРАВАЯ ЧАСТЬ: Округление и результаты
        # Блок выбора метода округления
        rounding_frame = tk.LabelFrame(
            right_frame, 
            text="Метод округления итога", 
            font=("Arial", 12, "bold"),
            width=280
        )
        rounding_frame.pack(fill="x", pady=(0, 20))
        
        rounding_methods = [
            ("Математическое\n2.5 → 3, -2.5 → -3", "mathematical"),
            ("Бухгалтерское\n2.5 → 2, 3.5 → 4", "bankers"),
            ("Усечение\n2.9 → 2, -2.9 → -2", "truncate")
        ]
        
        for i, (text, value) in enumerate(rounding_methods):
            rb_frame = tk.Frame(rounding_frame)
            rb_frame.pack(anchor="w", padx=15, pady=8, fill="x")
            
            tk.Radiobutton(
                rb_frame,
                variable=self.rounding_method_var,
                value=value,
                font=("Arial", 10)
            ).pack(side="left")
            
            tk.Label(
                rb_frame,
                text=text,
                font=("Arial", 10),
                justify="left"
            ).pack(side="left", padx=8)
        
        # Блок результатов
        results_frame = tk.LabelFrame(
            right_frame, 
            text="Результаты вычислений", 
            font=("Arial", 12, "bold"),
            width=280
        )
        results_frame.pack(fill="x", pady=(0, 20))
        
        # Основной результат
        tk.Label(
            results_frame, 
            text="Основной результат:", 
            font=("Arial", 11, "bold")
        ).pack(anchor="w", padx=15, pady=(10, 5))
        
        self.result_display = tk.Label(
            results_frame,
            textvariable=self.result_var,
            font=("Courier New", 12),
            bg="lightyellow",
            relief="solid",
            height=3,
            wraplength=250,
            justify="left"
        )
        self.result_display.pack(fill="x", padx=15, pady=(0, 15))
        
        # Округленный результат
        tk.Label(
            results_frame, 
            text="Округленный результат:", 
            font=("Arial", 11, "bold")
        ).pack(anchor="w", padx=15, pady=(0, 5))
        
        self.rounded_result_display = tk.Label(
            results_frame,
            textvariable=self.rounded_result_var,
            font=("Courier New", 14, "bold"),
            bg="#E6F7FF",
            relief="solid",
            height=2,
            wraplength=250,
            justify="center"
        )
        self.rounded_result_display.pack(fill="x", padx=15, pady=(0, 10))
        
        # Инструкция внизу
        instruction_frame = tk.LabelFrame(
            self.root, 
            text="Инструкция и информация", 
            font=("Arial", 11, "bold")
        )
        instruction_frame.pack(pady=10, padx=10, fill="x")
        
        instructions = [
            f"• Формула: Число1 [Оп1] (Число2 [Оп2] Число3) [Оп3] Число4",
            f"• Порядок вычислений: 1) (2 и 3) 2) × ÷ 3) + -",
            f"• Диапазон чисел: от {self.format_number_with_spaces('-1500000000000.0000000000')}",
            f"                    до {self.format_number_with_spaces('1500000000000.0000000000')}",
            f"• Промежуточные вычисления округляются до 10 знаков",
            f"• Поддерживается ввод с пробелами или без них",
            f"• Ctrl+C/Ctrl+V работают в любой раскладке клавиатуры"
        ]
        
        for instruction in instructions:
            label = tk.Label(
                instruction_frame,
                text=instruction,
                font=("Arial", 9),
                anchor="w",
                justify="left"
            )
            label.pack(anchor="w", padx=15, pady=2)
    
    def setup_bindings(self):
        """Настройка привязок клавиш для всех полей ввода"""
        for i, entry in enumerate(self.num_entries):
            entry.bind('<KeyRelease>', lambda e, idx=i: self.validate_input(e, idx))
            entry.bind('<FocusOut>', lambda e, idx=i: self.format_input_on_focusout(e, idx))
            # Привязываем обработчик Ctrl+KeyPress
            entry.bind('<Control-KeyPress>', self.handle_ctrl_key)
    
    def handle_ctrl_key(self, event):
        """Обработка Ctrl+Клавиша независимо от раскладки"""
        # Проверяем, что нажат Ctrl (0x4 = Control)
        if event.state & 0x4:
            keycode = event.keycode
            
            # Код 67 = C (латинская) / С (русская) - для копирования
            # Код 86 = V (латинская) / М (русская) - для вставки
            if keycode == 67:  # C/С - копирование
                self.copy_to_clipboard(event.widget)
                return "break"
            elif keycode == 86:  # V/М - вставка
                self.paste_from_clipboard(event.widget)
                return "break"
    
    def copy_to_clipboard(self, widget):
        """Копирование текста в буфер обмена"""
        if widget.selection_present():
            selected_text = widget.selection_get()
            self.root.clipboard_clear()
            self.root.clipboard_append(selected_text)
    
    def paste_from_clipboard(self, widget):
        """Вставка текста из буфера обмена с очисткой"""
        try:
            clipboard_text = self.root.clipboard_get()
        except:
            return
        
        # Определяем индекс поля ввода
        try:
            idx = self.num_entries.index(widget)
        except ValueError:
            return
        
        # Очистка вставляемого текста
        cleaned = re.sub(r'[^\d\-\.,\s]', '', clipboard_text)
        cleaned = cleaned.replace(',', '.')
        cleaned = re.sub(r'\s+', ' ', cleaned).strip()
        
        if cleaned.count('-') > 1:
            cleaned = cleaned[0] + cleaned[1:].replace('-', '')
        
        if cleaned.count('.') > 1:
            parts = cleaned.split('.')
            cleaned = parts[0] + '.' + ''.join(parts[1:])
        
        # Получаем текущий текст
        current_text = self.num_vars[idx].get()
        
        # Вставка текста
        if widget.selection_present():
            # Заменяем выделенный текст
            start = widget.index("sel.first")
            end = widget.index("sel.last")
            new_text = current_text[:start] + cleaned + current_text[end:]
        else:
            # Вставляем в позицию курсора
            cursor_pos = widget.index(tk.INSERT)
            new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
        
        self.num_vars[idx].set(new_text)
        
        # Вызываем валидацию
        self.validate_input(None, idx)
        
        # Ставим курсор после вставленного текста
        if widget.selection_present():
            new_cursor_pos = widget.index("sel.first") + len(cleaned)
        else:
            new_cursor_pos = widget.index(tk.INSERT) + len(cleaned)
        widget.icursor(new_cursor_pos)
    
    def validate_input(self, event, idx):
        """Валидация ввода в реальном времени"""
        entry_widget = self.num_entries[idx]
        error_var = self.error_vars[idx]
        text = self.num_vars[idx].get()
        
        # Сбрасываем ошибку
        error_var.set("")
        if idx in [1, 2]:  # Приоритетные поля
            entry_widget.config(bg="#FFF8DC")
        else:
            entry_widget.config(bg="white")
        
        if not text:
            return
        
        # Проверяем на несколько знаков плюса
        if text.count('+') > 1:
            error_var.set("Некорректный ввод: несколько знаков +")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Проверяем на несколько точек/запятых
        dot_count = text.count('.') + text.count(',')
        if dot_count > 1:
            error_var.set("Некорректный ввод: несколько разделителей")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Проверяем положение минуса
        minus_positions = [i for i, char in enumerate(text) if char == '-']
        if minus_positions:
            if minus_positions[0] != 0:
                error_var.set("Некорректный ввод: минус должен быть только в начале")
                entry_widget.config(bg="#FFE6E6")
                return
            if len(minus_positions) > 1:
                error_var.set("Некорректный ввод: несколько знаков -")
                entry_widget.config(bg="#FFE6E6")
                return
        
        # Проверяем минус после точки/запятой
        if '.' in text or ',' in text:
            separator = '.' if '.' in text else ','
            parts = text.split(separator)
            if len(parts) > 1 and '-' in parts[1]:
                error_var.set("Некорректный ввод: минус после разделителя")
                entry_widget.config(bg="#FFE6E6")
                return
        
        # Проверяем пробелы (если они есть)
        space_valid, space_error = self.check_spaces(text)
        if not space_valid:
            error_var.set(f"Некорректный ввод: {space_error}")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Заменяем запятую на точку
        text = text.replace(',', '.')
        
        # Проверяем на допустимые символы
        allowed_pattern = r'^[-]?[\d\s]*\.?[\d\s]*$'
        if not re.match(allowed_pattern, text):
            error_var.set("Некорректный ввод: недопустимые символы")
            entry_widget.config(bg="#FFE6E6")
            return
        
        # Удаляем все пробелы для дальнейшей обработки
        text_no_spaces = text.replace(' ', '')
        
        # Проверяем случаи типа ".123" или "-.123"
        if text_no_spaces.startswith('.') or text_no_spaces.startswith('-.'):
            pass
        
        # Проверяем длину целой и дробной части
        if '.' in text_no_spaces:
            integer_part, fractional_part = text_no_spaces.split('.')
            integer_part_for_check = integer_part.lstrip('-')
            
            if integer_part_for_check and len(integer_part_for_check) > 14:
                error_var.set("Слишком длинная целая часть (макс. 14 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
            
            if len(fractional_part) > 10:
                error_var.set("Слишком длинная дробная часть (макс. 10 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
        else:
            integer_part_for_check = text_no_spaces.lstrip('-')
            if integer_part_for_check and len(integer_part_for_check) > 14:
                error_var.set("Слишком длинная целая часть (макс. 14 цифр)")
                entry_widget.config(bg="#FFE6E6")
                return
    
    def check_spaces(self, text):
        """Проверка корректности пробелов-разделителей (если они есть)"""
        if not text:
            return True, ""
        
        if ' ' not in text:
            return True, ""
        
        if '  ' in text:
            return False, "Несколько пробелов подряд"
        
        check_text = text.lstrip('-')
        
        if '.' in check_text or ',' in check_text:
            separator = '.' if '.' in check_text else ','
            integer_part = check_text.split(separator)[0]
            fractional_part = check_text.split(separator)[1] if len(check_text.split(separator)) > 1 else ""
            
            if ' ' in fractional_part:
                return False, "Пробелы в дробной части недопустимы"
        else:
            integer_part = check_text
            fractional_part = ""
        
        integer_without_spaces = integer_part.replace(' ', '')
        
        if integer_without_spaces and not integer_without_spaces.isdigit():
            return False, "Недопустимые символы в целой части"
        
        return True, ""
    
    def format_input_on_focusout(self, event, idx):
        """Форматирование ввода при потере фокуса"""
        text = self.num_vars[idx].get()
        if not text:
            return
        
        text_no_spaces = text.replace(' ', '')
        text_no_spaces = text_no_spaces.replace(',', '.')
        
        formatted_text = self.format_number_with_spaces(text_no_spaces)
        self.num_vars[idx].set(formatted_text)
    
    def format_number_with_spaces(self, text):
        """Форматирование числа с пробелами-разделителями"""
        if not text:
            return ""
        
        sign = "-" if text.startswith('-') else ""
        text = text.lstrip('-')
        
        if text.startswith('.'):
            return sign + text
        
        if '.' in text:
            integer_part, fractional_part = text.split('.')
        else:
            integer_part, fractional_part = text, ""
        
        if integer_part:
            formatted_integer = ""
            for i, digit in enumerate(reversed(integer_part)):
                if i > 0 and i % 3 == 0:
                    formatted_integer = ' ' + formatted_integer
                formatted_integer = digit + formatted_integer
            
            result = sign + formatted_integer
        else:
            result = sign + "0"
        
        if fractional_part:
            result += '.' + fractional_part
        
        return result
    
    def parse_number(self, num_str):
        """Парсинг числа из строки"""
        if not num_str:
            return None, "Пустой ввод"
        
        num_str = num_str.replace(' ', '')
        num_str = num_str.replace(',', '.')
        
        try:
            if not re.match(r'^-?\d*\.?\d*$', num_str):
                return None, "Некорректный формат числа"
            
            num_str = num_str.strip()
            
            if num_str.startswith('.') or num_str.startswith('-.'):
                num_str = '0' + num_str if num_str.startswith('.') else '-0' + num_str[1:]
            
            if not num_str or num_str == '-':
                return Decimal('0'), ""
            
            num = Decimal(num_str)
            
            if num < self.MIN_VALUE or num > self.MAX_VALUE:
                return None, f"Число вне диапазона"
            
            return num, ""
        except (InvalidOperation, ValueError) as e:
            return None, f"Ошибка преобразования: {str(e)}"
    
    def is_overflow(self, num):
        """Проверка на переполнение диапазона"""
        return num < self.MIN_VALUE or num > self.MAX_VALUE
    
    def round_intermediate(self, num):
        """Округление промежуточного результата до 10 знаков"""
        try:
            return num.quantize(Decimal('0.0000000000'), rounding=ROUND_HALF_UP)
        except:
            return num
    
    def round_final(self, num):
        """Округление финального результата до 6 знаков"""
        try:
            return num.quantize(Decimal('0.000001'), rounding=ROUND_HALF_UP)
        except:
            return num
    
    def round_to_integer(self, num, method):
        """Округление до целых по выбранному методу"""
        if method == "mathematical":
            return num.quantize(Decimal('1'), rounding=ROUND_HALF_UP)
        elif method == "bankers":
            return num.quantize(Decimal('1'), rounding=ROUND_HALF_EVEN)
        else:  # truncate
            if num >= 0:
                return num.quantize(Decimal('1'), rounding=ROUND_DOWN)
            else:
                return num.quantize(Decimal('1'), rounding=ROUND_UP)
    
    def format_result(self, number):
        """Форматирование результата для отображения"""
        if number is None:
            return "Ошибка"
        
        if not isinstance(number, Decimal):
            try:
                number = Decimal(str(number))
            except:
                return "Ошибка формата"
        
        num_str = format(number, 'f')
        
        if '.' in num_str:
            integer_part, fractional_part = num_str.split('.')
        else:
            integer_part, fractional_part = num_str, "000000"
        
        sign = "-" if integer_part.startswith('-') else ""
        integer_part = integer_part.lstrip('-')
        
        if not integer_part:
            integer_part = "0"
        
        formatted_integer = ""
        for i, digit in enumerate(reversed(integer_part)):
            if i > 0 and i % 3 == 0:
                formatted_integer = ' ' + formatted_integer
            formatted_integer = digit + formatted_integer
        
        if len(fractional_part) < 6:
            fractional_part = fractional_part.ljust(6, '0')
        elif len(fractional_part) > 6:
            fractional_part = fractional_part[:6]
        
        fractional_part = fractional_part.rstrip('0')
        
        result = sign + formatted_integer
        if fractional_part:
            result += '.' + fractional_part
        
        return result
    
    def format_integer_result(self, number):
        """Форматирование целого результата для отображения"""
        if number is None:
            return "Ошибка"
        
        if not isinstance(number, Decimal):
            try:
                number = Decimal(str(number))
            except:
                return "Ошибка формата"
        
        num_str = format(number, 'f').split('.')[0]
        
        sign = "-" if num_str.startswith('-') else ""
        integer_part = num_str.lstrip('-')
        
        if not integer_part:
            integer_part = "0"
        
        formatted_integer = ""
        for i, digit in enumerate(reversed(integer_part)):
            if i > 0 and i % 3 == 0:
                formatted_integer = ' ' + formatted_integer
            formatted_integer = digit + formatted_integer
        
        return sign + formatted_integer
    
    def calculate_operation(self, a, b, op):
        """Выполнение одной операции"""
        if op == "+":
            return a + b
        elif op == "-":
            return a - b
        elif op == "×":
            return a * b
        elif op == "÷":
            if b == 0:
                raise ZeroDivisionError("Деление на ноль")
            return a / b
        else:
            raise ValueError(f"Неизвестная операция: {op}")
    
    def calculate_with_priority(self, numbers, operations):
        """Вычисление с правильным порядком операций"""
        ops = operations.copy()
        nums = numbers.copy()
        
        # 1. Сначала вычисляем приоритетную часть (числа 2 и 3 всегда в скобках)
        priority_result = self.calculate_operation(nums[1], nums[2], ops[1])
        priority_result = self.round_intermediate(priority_result)
        
        if self.is_overflow(priority_result):
            return None, "ПЕРЕПОЛНЕНИЕ (приоритетная часть)"
        
        # Заменяем числа 2 и 3 на результат приоритетной части
        calc_nums = [nums[0], priority_result, nums[3]]
        calc_ops = [ops[0], ops[2]]
        
        # 2. Выполняем умножение и деление (если они есть)
        i = 0
        while i < len(calc_ops):
            if calc_ops[i] in ["×", "÷"]:
                result = self.calculate_operation(calc_nums[i], calc_nums[i+1], calc_ops[i])
                result = self.round_intermediate(result)
                
                if self.is_overflow(result):
                    return None, "ПЕРЕПОЛНЕНИЕ (умножение/деление)"
                
                calc_nums[i] = result
                calc_nums.pop(i+1)
                calc_ops.pop(i)
            else:
                i += 1
        
        # 3. Теперь выполняем сложение и вычитание (слева направо)
        final_result = calc_nums[0]
        for i in range(len(calc_ops)):
            result = self.calculate_operation(final_result, calc_nums[i+1], calc_ops[i])
            result = self.round_intermediate(result)
            
            if self.is_overflow(result):
                return None, "ПЕРЕПОЛНЕНИЕ (сложение/вычитание)"
            
            final_result = result
        
        return final_result, ""
    
    def calculate(self):
        """Выполнение вычислений с 4 операндами"""
        for error_var in self.error_vars:
            error_var.set("")
        
        for i, entry in enumerate(self.num_entries):
            if i in [1, 2]:
                entry.config(bg="#FFF8DC")
            else:
                entry.config(bg="white")
        
        numbers = []
        errors = []
        
        for i in range(4):
            num_str = self.num_vars[i].get()
            num, error = self.parse_number(num_str)
            
            if error:
                errors.append((i, error))
            numbers.append(num)
        
        if errors:
            for idx, error in errors:
                self.error_vars[idx].set(error)
                self.num_entries[idx].config(bg="#FFE6E6")
            return
        
        try:
            operations = [op_var.get() for op_var in self.operation_vars]
            final_result, error = self.calculate_with_priority(numbers, operations)
            
            if error:
                self.result_var.set(error)
                self.rounded_result_var.set(error)
                return
            
            display_result = self.round_final(final_result)
            formatted_result = self.format_result(display_result)
            self.result_var.set(formatted_result)
            
            rounding_method = self.rounding_method_var.get()
            rounded_integer = self.round_to_integer(display_result, rounding_method)
            
            formatted_rounded = self.format_integer_result(rounded_integer)
            self.rounded_result_var.set(formatted_rounded)
            
        except ZeroDivisionError as e:
            self.result_var.set("Ошибка: деление на ноль")
            self.rounded_result_var.set("Ошибка: деление на ноль")
            for i in range(3):
                if self.operation_vars[i].get() == "÷" and numbers[i+1] == 0:
                    self.error_vars[i+1].set("Деление на ноль")
                    self.num_entries[i+1].config(bg="#FFE6E6")
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка при вычислении: {str(e)}")
    
    def clear_all(self):
        """Очистка всех полей"""
        for var in self.num_vars:
            var.set("0")
        
        for var in self.operation_vars:
            var.set("+")
        
        self.result_var.set("")
        self.rounded_result_var.set("")
        
        for error_var in self.error_vars:
            error_var.set("")
        
        for i, entry in enumerate(self.num_entries):
            if i in [1, 2]:
                entry.config(bg="#FFF8DC")
            else:
                entry.config(bg="white")
        
        self.rounding_method_var.set("mathematical")
        self.num_entries[0].focus_set()

def main():
    root = tk.Tk()
    app = CalculatorApp(root)
    
    # Глобальные обработчики Ctrl+C/Ctrl+V
    def global_copy(event):
        """Обработка копирования независимо от раскладки"""
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            if widget.selection_present():
                selected_text = widget.selection_get()
                root.clipboard_clear()
                root.clipboard_append(selected_text)
            return "break"
    
    def global_paste(event):
        """Обработка вставки независимо от раскладки"""
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            # Проверяем, что это одно из полей ввода калькулятора
            if widget in app.num_entries:
                # Получаем индекс поля
                try:
                    idx = app.num_entries.index(widget)
                except ValueError:
                    return "break"
                
                try:
                    clipboard_text = root.clipboard_get()
                except:
                    return "break"
                
                # Очистка текста
                cleaned = re.sub(r'[^\d\-\.,\s]', '', clipboard_text)
                cleaned = cleaned.replace(',', '.')
                cleaned = re.sub(r'\s+', ' ', cleaned).strip()
                
                if cleaned.count('-') > 1:
                    cleaned = cleaned[0] + cleaned[1:].replace('-', '')
                
                if cleaned.count('.') > 1:
                    parts = cleaned.split('.')
                    cleaned = parts[0] + '.' + ''.join(parts[1:])
                
                current_text = app.num_vars[idx].get()
                
                if widget.selection_present():
                    start = widget.index("sel.first")
                    end = widget.index("sel.last")
                    new_text = current_text[:start] + cleaned + current_text[end:]
                else:
                    cursor_pos = widget.index(tk.INSERT)
                    new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
                
                app.num_vars[idx].set(new_text)
                app.validate_input(None, idx)
                
                if widget.selection_present():
                    new_cursor_pos = widget.index("sel.first") + len(cleaned)
                else:
                    new_cursor_pos = widget.index(tk.INSERT) + len(cleaned)
                widget.icursor(new_cursor_pos)
            
            return "break"
    
    # Регистрируем глобальные горячие клавиши по кодам клавиш
    # Это работает в любой раскладке
    root.bind_all('<Control-Key-c>', global_copy)
    root.bind_all('<Control-Key-C>', global_copy)
    root.bind_all('<Control-Key-v>', global_paste)
    root.bind_all('<Control-Key-V>', global_paste)
    
    # Альтернативный способ через KeyPress с проверкой кода клавиши
    def handle_ctrl_keypress(event):
        """Универсальный обработчик Ctrl+клавиша"""
        if event.state & 0x4:  # Ctrl нажат
            keycode = event.keycode
            
            if keycode == 67:  # C/С - копирование
                return global_copy(event)
            elif keycode == 86:  # V/М - вставка
                return global_paste(event)
    
    root.bind_all('<Control-KeyPress>', handle_ctrl_keypress)
    
    root.mainloop()

if __name__ == "__main__":
    main()